In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("steam-200k.csv")
df.isnull().sum()

151603712                     0
The Elder Scrolls V Skyrim    0
purchase                      0
1.0                           0
0                             0
dtype: int64

In [4]:
df.rename(columns= {"151603712":"user id","The Elder Scrolls V Skyrim":"game title","purchase":"purchase","1.0":"behavior_name","0":"value"}, inplace = True)
df.head()

,user id,game title,purchase,behavior_name,value
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


## we are going to suggest the games based on the number of hours he played and what sort of game

In [5]:
df.describe()

,user id,behavior_name,value
count,1.999990e+05,199999.000000,199999.0
mean,1.036556e+08,17.874468,0.0
std,7.208084e+07,138.057292,0.0
min,5.250000e+03,0.100000,0.0
25%,4.738420e+07,1.000000,0.0
50%,8.691201e+07,1.000000,0.0
75%,1.542309e+08,1.300000,0.0
max,3.099031e+08,11754.000000,0.0


#### we cannot simply suggest game for the 1 or 2 hours play time so will take the 10 hours as the key point

In [10]:
df_final = df.query("behavior_name>10")

In [11]:
df_final.rename(columns = {"behavior_name":"time"})
df_final.drop("value",axis=1,inplace= True)

G:\anakonda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
df_final.drop("purchase",axis = 1, inplace =True)

G:\anakonda\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
df_final.head()

,user id,game title,behavior_name
0,151603712,The Elder Scrolls V Skyrim,273.0
2,151603712,Fallout 4,87.0
4,151603712,Spore,14.9
6,151603712,Fallout New Vegas,12.1
68,59945701,Ultra Street Fighter IV,238.0


## now we will build the reccomendation system for the above data using KNN algorithm

In [19]:
pivot_knn = df_final.pivot_table(index= "game title", columns = "user id", values="behavior_name").fillna(0)
pivot_knn.head()

user id,5250,76767,86540,229911,298950,547685,554278,561758,577614,622362,...,307915393,308101403,308178753,308202228,308238255,308503685,308638385,308851621,309213952,309404240
game title,,,,,,,,,,,,,,,,,,,,,
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Labours of Hercules II The Cretan Bull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Labours of Hercules III Girl Power,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3DMark,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from scipy.sparse import csr_matrix
csr_knn = csr_matrix(pivot_knn)
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(metric = "cosine", algorithm = "brute")
nbrs.fit(csr_knn)

NearestNeighbors(algorithm='brute', metric='cosine')

## now we will build an another reccomendation system using the Corelation 

In [23]:
pivot_corr = df_final.pivot_table(index= "user id",columns="game title",values = "behavior_name").fillna(0)
pivot_corr.head()

game title,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),100% Orange Juice,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,3DMark,3DMark Vantage,4 Elements,7 Days to Die,8BitBoy,A Game of Thrones - Genesis,...,Zombie Army Trilogy,Zombie Panic Source,Zombies Monsters Robots,Zuma's Revenge,iBomber Defense Pacific,liteCam Game 100 FPS Game Capture,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,theHunter,theHunter Primal
user id,,,,,,,,,,,,,,,,,,,,,
5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### now we will combine the all two outputs from the two reccomendation systemns and will draw the things which is similar in the both of them

### consider the game named "7 Days to Die", we will draw the reccomendations for this game

In [28]:
corr_mat = pivot_corr["7 Days to Die"]
corr_out = pivot_corr.corrwith(corr_mat)
game_corr_df = pd.DataFrame(corr_out,columns= ["corelation"])
rec_corr = game_corr_df.sort_values(ascending = False,by= "corelation").head(5).index

In [47]:
distances,indices = nbrs.kneighbors(pivot_knn.iloc[7].values.reshape(1,-1))
rec_1 = []
for i in indices:
    x = pivot_knn.index[i].values
    rec_1.append(x)
rec_1 = list(rec_1)    
rec_knn = []
for i in range(5):
    rec_knn.append(rec_1[0][i])

In [49]:
for i in rec_knn:
    if i in rec_corr:
        print(i)

7 Days to Die
Empyrion - Galactic Survival
The Forest
Dragon's Prophet (EU)
Lambda Wars Beta
